In [2]:
import re
import numpy as np
import pandas as pd 
from konlpy.tag import Okt
import pyLDAvis
import pyLDAvis.gensim
from gensim import corpora 
from gensim import models
import gensim

In [3]:
def trans(rowdata,kingname):
    rmhdata = []
    for num1 in range(len(rowdata)):
        for num in range(len(rowdata[num1])):
            rmhdata.append(re.sub('[(一-龥)]','',rowdata[num1][num]))
    wdata =[]
    for i in range(len(rmhdata)):
        wdata.append(re.sub('-|○|/|。|=|\n','',rmhdata[i]))
    
    sdata =[]
    for i in range(len(wdata)):
        if wdata[i] != '':
            sdata.append(wdata[i])
    ssdata =[]
    for i in range(len(sdata)):
        if re.search('[가-힣]+',sdata[i]):
            ssdata.append(sdata[i])
    data =[]
    for i in range(len(ssdata)):
        if kingname not in ssdata[i]:
           data.append(ssdata[i])

    return data

In [4]:
data =[]
for i in range(0,1):
    txt1 = open('정종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '정종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '백성' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'정종실록'))

In [5]:
# 시리즈 만들기
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("정종실록"," "))
df=df.map(lambda x : x.replace("정종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [6]:
len(df)

61

In [7]:
okt = Okt()

In [8]:
#형태소로 분리
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','우리','서로','가지','모두'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [9]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [10]:
 # 문서-단어 행렬 만들어주는 함수.
def build_doc_term_mat(documents):

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


In [11]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	대간	0.009498128667473793
	만일	0.008345779962837696
	백성	0.006959959864616394
	반드시	0.006765557453036308
	대성	0.005564646329730749
	자가	0.005342762917280197
	서경	0.0049317642115056515
	더욱	0.004507033154368401
	백관	0.004354632459580898
	관원	0.00410959217697382


Topic ID: 1
	하윤	0.008337597362697124
	건문	0.0072356280870735645
	부처	0.0067486390471458435
	대답	0.00671633705496788
	백성	0.005865677259862423
	자가	0.005527275614440441
	하니	0.005323934368789196
	때문	0.005065285600721836
	어찌	0.004980895202606916
	달이	0.004768873564898968


Topic ID: 2
	백성	0.013998896814882755
	건문	0.0072831506840884686
	반드시	0.006993227172642946
	하니	0.006454054266214371
	어찌	0.005574422888457775
	자가	0.005150791257619858
	마음	0.005063412245362997
	하라	0.00466524250805378
	생각	0.004285941366106272
	이제	0.004250986035913229


Topic ID: 3
	백성	0.014835910871624947
	태상왕	0.010592709295451641
	존호	0.00984632596373558
	제사	0.008866233751177788
	예조	0.00668362807482481
	하늘	0.006638646125793457
	생각	0.00576399313285

In [12]:
# 주제 :  백성
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.029697 -0.052283       1        1  15.724619
10    -0.064746 -0.057202       2        1  14.469706
15     0.114196  0.006347       3        1  11.513307
8     -0.034104  0.054270       4        1  10.932396
17     0.055681  0.025236       5        1   8.452102
2      0.038915 -0.054492       6        1   4.788590
18    -0.025860 -0.066569       7        1   4.457065
14     0.039112 -0.002025       8        1   4.257385
11    -0.048346  0.018869       9        1   4.084801
0     -0.002665 -0.010643      10        1   4.073892
16    -0.006990  0.026959      11        1   3.267721
9      0.027682 -0.004468      12        1   2.819513
4     -0.060870  0.062108      13        1   2.805369
19    -0.054174  0.046477      14        1   2.168043
3     -0.002264  0.041291      15        1   1.861526
1     -0.017300 -0.059406      16        1   1.741355
5      0.001025  0.003497      17        1   1.591288
7      0.005242 -0.021109      18        1   0.916255
6      0.056860  0.025783      19        1   0.038231
12     0.008303  0.017359      20        1   0.036838, topic_info=     Term       Freq      Total Category  logprob  loglift
10     백성  72.000000  72.000000  Default  30.0000  30.0000
880   태백성  14.000000  14.000000  Default  29.0000  29.0000
1133   토전  14.000000  14.000000  Default  28.0000  28.0000
45     마음  39.000000  39.000000  Default  27.0000  27.0000
484    하늘  36.000000  36.000000  Default  26.0000  26.0000
...   ...        ...        ...      ...      ...      ...
355   스스로   0.004977  16.725552  Topic20  -6.2557  -0.2134
301    만세   0.004567  10.785200  Topic20  -6.3417   0.1394
1      건문   0.005517  48.826954  Topic20  -6.1528  -1.1819
180   반드시   0.005207  33.388985  Topic20  -6.2106  -0.8596
112    자가   0.004903  34.461086  Topic20  -6.2707  -0.9513

[1250 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1547      9  0.449921   가가
1826      6  0.786007   가물
1827      6  0.548920   가뭄
485       4  0.455181   가부
34        1  0.225950  가운데
...     ...       ...  ...
2349     11  0.548405   횡포
1148      3  0.457129   후손
1148      5  0.304753   후손
1912      7  0.383990   휘하
1505      8  0.596692   흉년

[1567 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 11, 16, 9, 18, 3, 19, 15, 12, 1, 17, 10, 5, 20, 4, 2, 6, 8, 7, 13])

In [13]:
# 주제 : 제사

In [14]:
data =[]
for i in range(0,1):
    txt1 = open('정종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '정종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '제사' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'정종실록'))

In [15]:
df=pd.Series(keywordtxt)
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("정종실록"," "))
df=df.map(lambda x : x.replace("정종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [16]:
len(df)

33

In [17]:
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','모두','서로'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [18]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]

In [19]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	북면	0.010169259272515774
	절제	0.009881233796477318
	침구	0.006913570687174797
	박주	0.005979269277304411
	왜적	0.005803871899843216
	건문	0.004876799415796995
	정안	0.004864830523729324
	조전	0.004778051748871803
	풍해도	0.004546354990452528
	생각	0.004546335432678461


Topic ID: 1
	군사	0.019174695014953613
	정안	0.016020186245441437
	하니	0.01329869870096445
	김첨	0.012419539503753185
	제사	0.012081100605428219
	문하부	0.01128549687564373
	공이	0.01024723518639803
	탄핵	0.008501197211444378
	건문	0.007544360589236021
	소경	0.005713867023587227


Topic ID: 2
	군사	0.02226382866501808
	정안	0.014172783121466637
	제사	0.012613988481462002
	다시	0.010280616581439972
	건문	0.00943832378834486
	하니	0.008999340236186981
	종묘	0.007904832251369953
	절제	0.006930958945304155
	공이	0.0066899582743644714
	불사	0.006437692791223526


Topic ID: 3
	군사	0.03630068153142929
	정안	0.01948198676109314
	명령	0.011076554656028748
	공이	0.009037762880325317
	절제	0.007327476982027292
	하니	0.007146142423152924
	어찌	0.0049465312622487545


In [30]:
# 주제 :  제사
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.036114 -0.022206       1        1  21.667177
3     -0.022313 -0.046118       2        1  16.522226
18    -0.090074  0.072537       3        1  10.215557
12    -0.008418 -0.039234       4        1   7.534925
8      0.166902  0.041567       5        1   6.869789
16    -0.020262 -0.044702       6        1   6.797793
2      0.027251 -0.030766       7        1   6.032314
1      0.031147 -0.046937       8        1   4.422435
11    -0.024206 -0.053856       9        1   3.926573
5      0.014977  0.025103      10        1   3.119716
6      0.064913  0.015409      11        1   2.971298
14    -0.047282  0.008641      12        1   2.544579
7     -0.035682  0.101985      13        1   2.406436
17     0.055280  0.039823      14        1   2.109160
10    -0.009723 -0.026299      15        1   1.231993
4     -0.004127 -0.060368      16        1   0.644905
9     -0.001623 -0.004164      17        1   0.521635
15    -0.003230 -0.020361      18        1   0.374739
0     -0.028883  0.043203      19        1   0.044924
13    -0.028536  0.046742      20        1   0.041830, topic_info=     Term       Freq      Total Category  logprob  loglift
559    정안  42.000000  42.000000  Default  30.0000  30.0000
20     제사  21.000000  21.000000  Default  29.0000  29.0000
61     군사  56.000000  56.000000  Default  28.0000  28.0000
9      절제  21.000000  21.000000  Default  27.0000  27.0000
682    공이  25.000000  25.000000  Default  26.0000  26.0000
...   ...        ...        ...      ...      ...      ...
476    상고   0.003198   3.731915  Topic20  -6.0841   0.7172
1093   국본   0.003382   5.065159  Topic20  -6.0281   0.4677
651    정사   0.003577   9.830979  Topic20  -5.9720  -0.1394
559    정안   0.004112  42.536190  Topic20  -5.8328  -1.4649
313    정치   0.003257   4.557629  Topic20  -6.0660   0.5354

[1144 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
110       1  0.266402    가지
110       2  0.177602    가지
110       3  0.088801    가지
110       4  0.088801    가지
110       6  0.088801    가지
...     ...       ...   ...
1285      1  0.581535  훈신종친
22        1  0.197172    휘하
22        2  0.197172    휘하
22        6  0.197172    휘하
59       14  0.755795    흉년

[859 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 4, 19, 13, 9, 17, 3, 2, 12, 6, 7, 15, 8, 18, 11, 5, 10, 16, 1, 14])

In [31]:
# 주제 : 상소

In [32]:
data =[]
for i in range(0,1):
    txt1 = open('정종실록.txt',encoding='utf-8').readlines()
    date = []
    keywordtxt =[]
    for num,i in enumerate(txt1):
        if '정종실록' in i:
            if ('년'and '월' and '일') in i:
                date.append(num)

    for k in range(len(date)-1):
        count =0
        for j in range(date[k],date[k+1]):
            if '상소' in txt1[j]:
                count +=1
        if count >0:
            keywordtxt.append(txt1[date[k]:date[k+1]])
    data.append(trans(keywordtxt,'정종실록'))

In [33]:
df=pd.Series(keywordtxt)
df=df.map(lambda x : re.sub('[^ ㄱ-ㅣ가-힣]+','',str(x)))
df=df.map(lambda x : x.replace("정종실록"," "))
df=df.map(lambda x : x.replace("정종"," "))
df=df.map(lambda x : x.replace("번째기사"," "))

In [34]:
len(df)

56

In [35]:
stopword = ['전하','임금','사람','여러','나라','로써','또한','지금','하니','이제','가지'
           ]
text=[]
for i in range(0,len(df)):
    t=okt.nouns(str(df[i]))
    temp =[]
    for j in t:
        if j not in stopword:
            if len(j)>1:
                temp.append(j)
    text.append(temp)

In [36]:
dictionary = corpora.Dictionary(text)
corpus = [dictionary.doc2bow(t) for t in text]
model = models.ldamodel.LdaModel(corpus, num_topics = 3, id2word = dictionary) 

In [37]:
NUM_TOPICS = 20

NUM_TOPIC_WORDS = 10

def print_topic_words(model):
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

corpus, dictionary = build_doc_term_mat(text)
model = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
print_topic_words(model)


Printing topic words.

Topic ID: 0
	문하부	0.01370491087436676
	상소	0.010507488623261452
	정사	0.008517450653016567
	대간	0.008457059040665627
	어찌	0.007359036710113287
	정지	0.0072842370718717575
	건문	0.006575096398591995
	기복	0.006523543503135443
	생각	0.006467579863965511
	설치	0.006093512289226055


Topic ID: 1
	문하부	0.015672864392399788
	상소	0.015118427574634552
	거둥	0.009993109852075577
	윤허	0.009975074790418148
	태상왕	0.009856630116701126
	모두	0.008288045413792133
	반드시	0.008261517621576786
	백성	0.008167867548763752
	비록	0.007452913094311953
	건문	0.007172895595431328


Topic ID: 2
	종친	0.009839632548391819
	문하부	0.009701965376734734
	부마	0.009112010709941387
	제도	0.007966568693518639
	모두	0.0077142370864748955
	보전	0.007628854364156723
	반드시	0.006967343855649233
	서경	0.006581687368452549
	탄핵	0.006537089589983225
	태상왕	0.006244059186428785


Topic ID: 3
	태상왕	0.02368401736021042
	문하부	0.010938040912151337
	상소	0.010205214843153954
	윤허	0.00979671347886324
	건문	0.00963770691305399
	성석린	0.008424860425293446
	제릉	0.00800378

In [38]:
pyLDAvis.enable_notebook()
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data 

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11     0.011496 -0.002715       1        1  9.952679
19     0.005404  0.045358       2        1  9.617249
7     -0.069405 -0.038404       3        1  9.310147
5     -0.034841 -0.048754       4        1  7.799706
4      0.082117  0.023943       5        1  7.518362
1     -0.042412  0.031737       6        1  6.913170
10     0.105822  0.001101       7        1  6.806604
14     0.025329 -0.069592       8        1  5.888548
8      0.003268 -0.016316       9        1  5.049895
9     -0.023338  0.000529      10        1  4.994555
3     -0.060521  0.097516      11        1  4.333847
2      0.004253 -0.004022      12        1  4.111633
0      0.015330 -0.039652      13        1  3.647914
6      0.032795  0.049189      14        1  3.571639
16     0.048752  0.010346      15        1  2.764745
12    -0.000979 -0.049485      16        1  2.401891
13    -0.024476 -0.018082      17        1  1.934859
17     0.007729  0.016233      18        1  1.793558
15    -0.025581  0.027836      19        1  0.828788
18    -0.060744 -0.016767      20        1  0.760226, topic_info=     Term       Freq      Total Category  logprob  loglift
77     탄핵  19.000000  19.000000  Default  30.0000  30.0000
343   태상왕  41.000000  41.000000  Default  29.0000  29.0000
26    문하부  56.000000  56.000000  Default  28.0000  28.0000
407    군사  24.000000  24.000000  Default  27.0000  27.0000
1049   토전  13.000000  13.000000  Default  26.0000  26.0000
...   ...        ...        ...      ...      ...      ...
54     처음   0.215163  20.349316  Topic20  -5.5202   0.3299
180    비록   0.217978  26.844275  Topic20  -5.5072   0.0659
31     상소   0.231478  65.580933  Topic20  -5.4472  -0.7672
368   반드시   0.219555  39.061337  Topic20  -5.5000  -0.3020
26    문하부   0.211642  56.733273  Topic20  -5.5367  -0.7119

[1263 rows x 6 columns], token_table=      Topic      Freq Term
term                      
213      11  0.692448   가기
756      10  0.453069   가끔
156       6  0.718627   가선
214      11  0.691770  가을로
380       1  0.051255   각각
...     ...       ...  ...
1064      3  0.389444   후회
755       2  0.214071   훈신
755       4  0.214071   훈신
755      16  0.214071   훈신
1168      5  0.567253   휴가

[1912 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 20, 8, 6, 5, 2, 11, 15, 9, 10, 4, 3, 1, 7, 17, 13, 14, 18, 16, 19])